# Notebook for submission

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
#import amp_pd_peptide
from src.model import LSTMmodel
from src.utils import get_patient_visits

/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.16.0-unknown is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.12.1-git20200711.33e2d80-dfsg1-0.6 is an invalid version and will not be supported in a future release
  warnings.warn(
/home/mariano/.local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_cuda.so: cannot open shared object file: No such file or directory'If 

In [3]:
checkpoint_path = 'model.ckpt'
model = LSTMmodel()
#model.load_from_checkpoint(checkpoint_path)
model.eval()

LSTMmodel(
  (lstm): LSTM(8, 128, batch_first=True)
  (linear): Linear(in_features=128, out_features=16, bias=True)
)

In [ ]:
env = amp_pd_peptide.make_env()   # initialize the environme
train = pd.read_csv('parkinsonPrediction/data/train_clinical_data.csv')
sup = pd.read_csv('parkinsonPrediction/data/supplemental_clinical_data.csv')

# Empty dataframe 
sample_prediction_df = pd.DataFrame(columns=['prediction_id', 'rating', 'group_key'])
iter_test = env.iter_test()    # an iterator which loops over the test files

for (test, test_peptides, test_proteins, sample_submission) in iter_test:
    for index, row in test.iterrows():
        # Get patient id
        patient_id = row['patient_id']
        # Get group key
        group_key = row['group_key']
        # Get updrs test 
        updrs_test = row['updrs_test']
        # Get last visits
        last_visits = get_patient_visits(patient_id, train, sup)
        # Get prediction
        pred = model.predict(last_visits)
        # Build prediction id (patient_id + group_key + updrs_test + months) 3342_0_updrs_1_plus_0_months
        pred_id0 = str(patient_id) + '_' + str(group_key) + '_' + str(updrs_test) + '_plus_0_months'
        pred_id1 = str(patient_id) + '_' + str(group_key) + '_' + str(updrs_test) + '_plus_6_months'
        pred_id2 = str(patient_id) + '_' + str(group_key) + '_' + str(updrs_test) + '_plus_12_months' 
        pred_id3 = str(patient_id) + '_' + str(group_key) + '_' + str(updrs_test) + '_plus_24_months' 

        # Append prediction to dataframe (prediction_id,rating,group_key)
        sample_prediction_df = sample_prediction_df.append({'prediction_id': pred_id0, 'rating': pred[0], 'group_key': group_key}, ignore_index=True)
        sample_prediction_df = sample_prediction_df.append({'prediction_id': pred_id1, 'rating': pred[1], 'group_key': group_key}, ignore_index=True)
        sample_prediction_df = sample_prediction_df.append({'prediction_id': pred_id2, 'rating': pred[2], 'group_key': group_key}, ignore_index=True)
        sample_prediction_df = sample_prediction_df.append({'prediction_id': pred_id3, 'rating': pred[3], 'group_key': group_key}, ignore_index=True)

    env.predict(sample_prediction_df)
    

In [20]:
import torch
train = 'data/train_clinical_data.csv'
sup = 'data/supplemental_clinical_data.csv'
test = pd.read_csv('data/example_test_files/test.csv')
patient_id = 55
last_visits = get_patient_visits(patient_id, train, sup)

last_visits = torch.from_numpy(last_visits).float()
last_visits = last_visits.unsqueeze(0)
pred = model.predict(last_visits)
last_visits.shape, pred.shape

(torch.Size([1, 10, 8]), torch.Size([1, 4, 4]))